In [1]:
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

In [2]:
from sklearn.datasets import load_iris, load_digits, load_wine, load_breast_cancer, load_diabetes

classification_loaders = {
    "iris": load_iris,
    "digits": load_digits,
    "wine": load_wine,
    "breast_cancer": load_breast_cancer
}

regression_loaders = {
    "diabetes": load_diabetes,
}

In [3]:
from sklearn import tree
from decision_tree import DecisionTreeClassifier, DecisionTreeRegressor

classification_models = {
    "sklearn": tree.DecisionTreeClassifier(criterion="entropy", max_depth=4),
    "ours": DecisionTreeClassifier(max_depth=4, min_samples=1)
}

regression_models = {
    "sklearn": tree.DecisionTreeRegressor(criterion="squared_error", max_depth=4),
    "ours": DecisionTreeRegressor(max_depth=4, min_samples=1)
}

In [4]:
def benchmark_dataset(ds_name, loader, model):
    dataset = loader()
    X, y = dataset["data"], dataset["target"]
    cv_scores = []
    for train_idx, test_idx in KFold(n_splits=5, shuffle=True).split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        model.fit(X_train, y_train)
        cv_scores.append(model.score(X_test, y_test))
    cv_score = np.mean(cv_scores)
    return cv_score


In [5]:
from prettytable import PrettyTable

def benchmark(loaders, models):
    leaderboard = PrettyTable()
    dataset_names = list(loaders.keys())
    leaderboard.add_column("Dataset", dataset_names)

    for model_name, model in models.items():
        results = [
            f"{benchmark_dataset(ds_name, loaders[ds_name], model):.3f}"
            for ds_name in dataset_names
        ]
        leaderboard.add_column(model_name, results)

    print(leaderboard)

In [6]:
benchmark(classification_loaders, classification_models)

+---------------+---------+-------+
|    Dataset    | sklearn |  ours |
+---------------+---------+-------+
|      iris     |  0.947  | 0.947 |
|     digits    |  0.651  | 0.694 |
|      wine     |  0.966  | 0.949 |
| breast_cancer |  0.931  | 0.930 |
+---------------+---------+-------+


In [7]:
benchmark(regression_loaders, regression_models)

+----------+---------+-------+
| Dataset  | sklearn |  ours |
+----------+---------+-------+
| diabetes |  0.277  | 0.313 |
+----------+---------+-------+
